# Testing with the synchronous API

In [45]:
import openai
import os
import base64

# Load the image
with open("regularized_data/1.jpg", "rb") as image_file:
    image = image_file.read()

# Encode the image to base64
encoded_image = base64.b64encode(image).decode("utf-8")

# Initialization of OpenAI client with your environment API key
openai.api_key = os.environ.get("OPENAI_API_KEY", "your-api-key")

# Construct the prompt message for the model
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "You are a UI designer generating description for icons to be added as accessibility description for usage in websites and apps , you are to generate 5 captions covering diverse interpretations of the icon based on different UI contexts. The output should be of this following json format : { captions : [ caption1 , caption2 , ... caption5 ] } ",
            {"image": encoded_image},
        ],
    },
]

# Parameters for the API request
params = {
    "model": "gpt-4o",  # Make sure you have access to this model
    "messages": PROMPT_MESSAGES,
    "temperature":0.1,
    "response_format":{ 
        "type": "json_object"
    },
    "max_tokens": 200,
}

try:
    result = openai.ChatCompletion.create(**params)
    print(result.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")


{
  "captions": [
    "Completed tasks list",
    "Checklist with a completed item",
    "Verified document",
    "Task list with a checkmark",
    "Approved items"
  ]
}


In [46]:
# Parameters for the API request
params = {
    "model": "gpt-4o-mini",  # Make sure you have access to this model
    "messages": PROMPT_MESSAGES,
    "temperature":0.1,
    "response_format":{ 
        "type": "json_object"
    },
    "max_tokens": 200,
}

try:
    result = openai.ChatCompletion.create(**params)
    print(result.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {e}")

{
  "captions": [
    "A checklist icon indicating completed tasks or items.",
    "An icon representing a list with a checkmark, suggesting verification or approval.",
    "A symbol for organized information, highlighting items that have been confirmed.",
    "An icon depicting a series of lines with a check, indicating a successful completion of a process.",
    "A visual cue for task management, showing a list with a tick to denote finished actions."
  ]
}


# Using the batch API in limits

Before running this clean up all the stuff above ( restart kernal ) there will be a name space collision with how openai package is imported in the two versions

In [1]:
# Cell 1: Import Libraries and Setup

import os
import base64
import json
from openai import OpenAI

# Initialize your OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "your-api-key"))

# Specify the image directory and model name
image_directory = "regularized_data"
model_name = "gpt-4o"


In [2]:
# Cell 2: Function to Prepare JSONL File

def prepare_jsonl(image_dir, max_images):
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Apply limit to the number of images processed
    limit = min(max_images, len(image_files))
    selected_images = image_files[:limit]

    # Prepare a JSONL file
    json_lines = []
    for filename in selected_images:
        file_path = os.path.join(image_dir, filename)
        with open(file_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

        prompt_messages = [
            {"role": "system", "content":"You are a UI designer generating description for icons to be added as accessibility description for usage in websites and apps , you are to generate 5 captions covering diverse interpretations of the icon based on different UI contexts. The output should be of this following json format : { captions : [ caption1 , caption2 , ... caption5 ] } " },
            {"role": "user", "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{encoded_image}"}},]}
        ]

        json_line = {
            "custom_id": filename,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": model_name,
                "messages": prompt_messages,
                "max_tokens": 1000,
                "temperature": 0.1,
                "response_format":{ 
                    "type": "json_object"
                },
            }
        }
        json_lines.append(json_line)

    # Write to JSONL file
    output_jsonl_file = "batch_input.jsonl"
    with open(output_jsonl_file, "w") as f:
        for line in json_lines:
            f.write(json.dumps(line) + "\n")

    print(f"JSONL file '{output_jsonl_file}' created with {len(json_lines)} entries.")


In [3]:
# Cell 3: Upload JSONL File

def upload_jsonl(file_path):
    try:
        batch_input_file = client.files.create(file=open(file_path, "rb"), purpose="batch")
        print(f"File uploaded with ID: {batch_input_file.id}")
        return batch_input_file.id
    except Exception as e:
        print(f"Uploading failed: {e}")
        return None


In [4]:
# Cell 4: Create a Batch

def create_batch(file_id):
    try:
        batch_job = client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch created with ID: {batch_job.id}")
        return batch_job.id
    except Exception as e:
        print(f"Creating batch failed: {e}")
        return None


In [5]:
# Cell 5: Check Batch Status

def check_batch_status(batch_id):
    try:
        batch_job = client.batches.retrieve(batch_id)
        print(f"Batch status: {batch_job.status}")
        return batch_job
    except Exception as e:
        print(f"Error checking status: {e}")
        return None


In [6]:
# Cell 6: Download Results

def download_results(file_id):
    try:
        file_response = client.files.content(file_id)
        return file_response
    except Exception as e:
        print(f"Error downloading results: {e}")
        return None


In [7]:
# Cell 7a: Specify the Maximum Number of Images

# Calculate the total number of images
num_images = len([f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

# Set max_images to the total number of available images
max_images = num_images  # This will set the limit to all image files available

In [8]:
# Cell 7a: Specify the Maximum Number of Images

# Set the maximum number of images to process in this JSONL file
max_images = 1   # Adjust this number based on your testing needs


In [9]:
# Cell 7b: Prepare the JSONL File

prepare_jsonl(image_directory, max_images)


JSONL file 'batch_input.jsonl' created with 1 entries.


In [10]:
# Cell 7c: Upload the JSONL File

file_id = upload_jsonl("batch_input.jsonl")


File uploaded with ID: file-ITN76dJDm3JppCJ4aTU9Uolk


In [11]:
# Cell 7d: Create a Batch

batch_id = None
if file_id:
    batch_id = create_batch(file_id)


Batch created with ID: batch_dPoFRu7n1hMYHSXS3gj9SqGQ


In [13]:
# Cell 7e: Manually Check the Batch Status

if batch_id:
    batch_status = check_batch_status(batch_id)

print(batch_status)


Batch status: completed
Batch(id='batch_dPoFRu7n1hMYHSXS3gj9SqGQ', completion_window='24h', created_at=1724251803, endpoint='/v1/chat/completions', input_file_id='file-ITN76dJDm3JppCJ4aTU9Uolk', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1724251809, error_file_id=None, errors=None, expired_at=None, expires_at=1724338203, failed_at=None, finalizing_at=1724251808, in_progress_at=1724251804, metadata=None, output_file_id='file-PzrIyvcRLQEdzjqDA8ekQSgs', request_counts=BatchRequestCounts(completed=1, failed=0, total=1))


In [14]:
# Function to Download Error Details
def download_error_file(file_id):
    try:
        # Get content from the error file
        file_content = client.files.content(file_id)
        print("Error file content:")
        print(file_content.content)
        return file_content.content
    except Exception as e:
        print(f"Error downloading error file: {e}")
        return None

# Use this function with the error file ID
if batch_status is not None and batch_status.error_file_id:
    error_file_content = download_error_file(batch_status.error_file_id)
else:
    print("No error file ID available.")


No error file ID available.


In [15]:
# Cell 7f: Download Results if Completed

if batch_status and batch_status.status == "completed":
    output_file_id = batch_status.output_file_id
    results = download_results(output_file_id)
    
    # Save the result to captions.jsonl
    with open("captions.jsonl", "w") as f:
        f.write(results.text)
    print("Captions saved to captions.jsonl")
else:
    print(f"Batch status is {batch_status.status}. Check again later.")


Captions saved to captions.jsonl
